# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Nguyễn Thuận Phát
- MSSV: 21127665
- Lớp: 21CLC08

# Import

In [1398]:
import pandas as pd
import numpy as np
import math #làm tròn số

# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [1399]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần
X_train_1a = train.iloc[:, :11]
X_test_1a = test.iloc[:, :11]
dataset_1b = train.iloc[:, 18:24]#5 cột yêu cầu và cột salary
dataset_1c = pd.concat([train.iloc[:, 7:10], train.iloc[:, -1]], axis = 1)#3 cột yêu cầu và cột salary
train_Quant_col = train[['Quant']]
train_10percentage_col = train[['10percentage']]
train_12percentage_col = train[['12percentage']]
train_Salary_col = train[['Salary']]
X_train_1d_1 = train[['Quant', '10percentage', '12percentage', 'Logical', 'collegeGPA', 'English', 'CollegeTier', 'Degree', 'Gender', 'ComputerProgramming']]
X_test_1d_1 = test[['Quant', '10percentage', '12percentage', 'Logical', 'collegeGPA', 'English', 'CollegeTier', 'Degree', 'Gender', 'ComputerProgramming']]

# Cài đặt hàm

In [1400]:
# Cài đặt các hàm cần thiết ở đây

#các hàm, class lấy từ lab04.ipynb
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self


    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1) 

def k_foldCrossValidation(dataset, k = 10):
    foldSize = math.ceil(dataset.shape[0] / k)
    shuffledDataset = dataset.sample(frac = 1)#xáo trộn dữ liệu

    #chia dataset thành k fold
    folds = []
    for i in range(k):
        folds.append(shuffledDataset.iloc[i * foldSize : i * foldSize + foldSize])
    
    mae_list = []
    #duyệt lần lượt từng fold
    for current in range(len(folds)):
        #fold hiện tại dùng để test
        fold_X_test = folds[current].iloc[:, :-1].to_numpy()
        fold_y_test = folds[current].iloc[:, -1].to_frame().to_numpy()

        #ghép các folds còn lại thành train dataset
        otherFolds = []
        for other in range(len(folds)):
            if other != current:
                otherFolds.append(folds[other])
        otherFolds = pd.concat([f for f in otherFolds])

        fold_X_train = otherFolds.iloc[:, :-1].to_numpy()
        fold_y_train = otherFolds.iloc[:, -1].to_frame().to_numpy()

        fold_lr = OLSLinearRegression().fit(fold_X_train, fold_y_train)
        fold_pred = fold_lr.predict(fold_X_test)

        #lưu độ lỗi các lần lặp vào 1 list rồi sau đó tính trung bình để đánh giá đặc trưng
        mae_list.append(mae(fold_y_test, fold_pred))

        #hủy mô hình 
        fold_lr = 0

    return np.mean(mae_list, axis = 0)
    

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [1401]:
# Phần code cho yêu cầu 1a
lr_1a =  OLSLinearRegression().fit(X_train_1a, y_train)
params_1a = round(lr_1a.get_params(), 3)
y_pred_1a = lr_1a.predict(X_test_1a)
print("11 hệ số ứng với từng đặc trưng:")
for i in range(0, 11):
    print(X_train_1a.columns[i], ":", params_1a.values[i])

11 hệ số ứng với từng đặc trưng:
Gender : -22756.513
10percentage : 804.503
12percentage : 1294.655
CollegeTier : -91781.898
Degree : 23182.389
collegeGPA : 1437.549
CollegeCityTier : -8570.662
English : 147.858
Logical : 152.888
Quant : 117.222
Domain : 34552.286


In [1402]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae_1a = round(mae(y_test, y_pred_1a), 3)
print("Độ lỗi tuyệt đối trung bình MAE: ", mae_1a)

Độ lỗi tuyệt đối trung bình MAE:  104863.778


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 
- 22756.513 \times \text{Gender}
+ 804.503 \times \text{10percentage}
+ 1294.655 \times \text{12percentage}
- 91781.898 \times \text{CollegeTier}
+ 23182.389 \times \text{Degree}
+ 1437.549 \times \text{collegeGPA}
- 8570.662 \times \text{CollegeCityTier}
+ 147.858 \times \text{English}
+ 152.888 \times \text{Logical}
+ 117.222 \times \text{Quant}
+ 34552.286 \times \text{Domain}$$
$$\text{MAE} = 104863.778$$

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 
- 22756.513 \times \text{Gender}
+ 804.503 \times \text{10percentage}
+ 1294.655 \times \text{12percentage}
- 91781.898 \times \text{CollegeTier}
+ 23182.389 \times \text{Degree}
+ 1437.549 \times \text{collegeGPA}
- 8570.662 \times \text{CollegeCityTier}
+ 147.858 \times \text{English}
+ 152.888 \times \text{Logical}
+ 117.222 \times \text{Quant}
+ 34552.286 \times \text{Domain}$$
$$\text{MAE} = 104863.778$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [1403]:
# Phần code cho yêu cầu 1b
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

result_1b = []
for col in dataset_1b.columns:
    if col == 'Salary':
        continue
    colData = dataset_1b[[col, 'Salary']]#lấy dữ liệu từng cột và cột salary
    result_1b.append([col, k_foldCrossValidation(colData, 5)])
    
#báo cáo kết quả
print("MAE trung bình của từng đặc trưng:")
for i in range(len(result_1b)):
    print(result_1b[i][0], ":", result_1b[i][1])

min = 0
for j in range(len(result_1b)):
    if result_1b[j][1] < result_1b[min][1]:
        min = j
print("Đặc trưng tốt nhất là:", result_1b[min][0])

MAE trung bình của từng đặc trưng:
conscientiousness : 306355.6813225526
agreeableness : 300840.82958532067
extraversion : 306880.51733153645
nueroticism : 299194.8697511968
openess_to_experience : 303019.4725539618
Đặc trưng tốt nhất là: nueroticism


In [1404]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
bpf_X_train = train.iloc[:, 21].to_frame().to_numpy()
bpf_y_train = train.iloc[:, -1].to_frame().to_numpy()
bpf_X_test = test.iloc[:, 21].to_frame().to_numpy()
bpf_y_test = test.iloc[:, -1]

bpf_lr = OLSLinearRegression().fit(bpf_X_train, bpf_y_train)
bpf_param = bpf_lr.get_params()
print("Hệ số:", round(bpf_param[0][0], 3))


Hệ số: -56546.304


In [1405]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
bpf_pred = bpf_lr.predict(bpf_X_test)
bpf_mae = round(mae(bpf_y_test, bpf_pred), 3)
print("Độ lỗi trung bình:", bpf_mae)

Độ lỗi trung bình: 291019.693


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -56546.304 \times \text{nueroticism}$$
$$\text{MAE} = 291019.693$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [1406]:
# Phần code cho yêu cầu 1c
# Tìm ra đặc trưng tốt nhất
# In ra các kết quả cross-validation như yêu cầu

#sử dụng lại code của câu 1b
result_1c = []
for col in dataset_1c.columns:
    if col == 'Salary':
        continue
    colData = dataset_1c[[col, 'Salary']]#lấy dữ liệu từng cột và cột salary
    result_1c.append([col, k_foldCrossValidation(colData, 10)])
    
#báo cáo kết quả
print("MAE trung bình của từng đặc trưng:")
for i in range(len(result_1c)):
    print(result_1c[i][0], ":", result_1c[i][1])

min = 0
for j in range(len(result_1c)):
    if result_1c[j][1] < result_1c[min][1]:
        min = j
print("Đặc trưng tốt nhất là:", result_1c[min][0])

MAE trung bình của từng đặc trưng:
English : 121877.00983056084
Logical : 120291.04847563757
Quant : 118066.19901927041
Đặc trưng tốt nhất là: Quant


In [1407]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện

bsf_X_train = train.iloc[:, 9].to_frame().to_numpy()
bsf_y_train = train.iloc[:, -1].to_frame().to_numpy()
bsf_X_test = test.iloc[:, 9].to_frame().to_numpy()
bsf_y_test = test.iloc[:, -1].to_numpy()

bsf_lr = OLSLinearRegression().fit(bsf_X_train, bsf_y_train)
bsf_param = bsf_lr.get_params()
print("Hệ số:", round(bsf_param[0][0], 3))


Hệ số: 585.895


In [1408]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_skill_feature_model

bsf_pred = bsf_lr.predict(bsf_X_test)
bsf_mae = round(mae(bsf_y_test, bsf_pred), 3)
print("Độ lỗi trung bình:", bsf_mae)

Độ lỗi trung bình: 106819.578


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 585.895 \times \text{Quant}$$
$$\text{MAE} = 106819.578$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [1409]:
# Trình bày các phần tìm ra mô hình

print("chỉ tính salary theo 1 đặc trưng(giống câu 1b, 1c nhưng sử dụng hết tất cả đặc trưng), sử dụng k-fold CV xem độ lỗi của từng đặc trưng để chọn")
#sử dụng lại code của câu 1b, 1c
result = []
for col in train.columns:
    if col == 'Salary':
        continue
    colData = train[[col, 'Salary']]
    result.append([col, k_foldCrossValidation(colData, 10)])
    
print("MAE trung bình của từng đặc trưng(đã sắp xếp tăng dần):")
result = pd.DataFrame(result, columns=['feature', 'avg MAE'])
result = result.sort_values(by = 'avg MAE')
print(result.to_string(index=False),"\n")

print("Mô hình 1: Sử dụng 10 đặc trưng có độ lỗi trung bình ít nhất đầu tiên ở trên")
print("Mô hình 2: sử dụng Quant bình phương, 10percentage bình phương")
print("Mô hình 3: sử dụng Quant bình phương, 10percentage bình phương, 12percentage bình phương")

chỉ tính salary theo 1 đặc trưng(giống câu 1b, 1c nhưng sử dụng hết tất cả đặc trưng), sử dụng k-fold CV xem độ lỗi của từng đặc trưng để chọn
MAE trung bình của từng đặc trưng(đã sắp xếp tăng dần):
              feature       avg MAE
                Quant 118115.851302
         10percentage 118781.081046
         12percentage 120005.763364
              Logical 120268.003125
           collegeGPA 121438.106011
              English 121904.005019
          CollegeTier 133445.151691
               Degree 137520.202884
               Gender 150360.193652
  ComputerProgramming 155965.986499
               Domain 175663.303270
      CollegeCityTier 251510.454447
ElectronicsAndSemicon 257418.401936
      ComputerScience 268470.591231
          TelecomEngg 290969.868837
       MechanicalEngg 297289.428745
          nueroticism 299282.692385
        agreeableness 300865.169458
       ElectricalEngg 301153.117407
openess_to_experience 303029.413161
            CivilEngg 306120.871302
    consc

## Thử nghiệm, so sánh các mô hình

In [1410]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu
print("Độ lỗi trung bình của các mô hình khi sử dụng k-fold Cross Validation:")
result_1d = []

dataset_1d_1 = train[['Quant', '10percentage', '12percentage', 'Logical', 'collegeGPA', 'English', 'CollegeTier', 'Degree', 'Gender', 'ComputerProgramming', 'Salary']]
result_1d.append(["Mô hình 1", k_foldCrossValidation(dataset_1d_1, 10)])

dataset_1d_2 = pd.concat((train_Quant_col**2, train_10percentage_col**2, train_Salary_col), axis = 1)
result_1d.append(["Mô hình 2", k_foldCrossValidation(dataset_1d_2, 10)])

dataset_1d_3 = pd.concat((train_Quant_col**2, train_10percentage_col**2, train_12percentage_col**2, train_Salary_col), axis = 1)
result_1d.append(["Mô hình 3", k_foldCrossValidation(dataset_1d_3, 10)])

print("MAE trung bình của từng mô hình:")
for i in range(len(result_1d)):
    print(result_1d[i][0], ":", result_1d[i][1])

min = 0
for j in range(len(result_1d)):
    if result_1d[j][1] < result_1d[min][1]:
        min = j
print("Mô hình tốt nhất là:", result_1d[min][0])

Độ lỗi trung bình của các mô hình khi sử dụng k-fold Cross Validation:
MAE trung bình của từng mô hình:
Mô hình 1 : 113016.48648784915
Mô hình 2 : 115840.03338759982
Mô hình 3 : 115386.4382907683
Mô hình tốt nhất là: Mô hình 1


In [1411]:
# Huấn luyện lại mô hình my_best_model trên toàn bộ tập huấn luyện
lr_1d = OLSLinearRegression().fit(X_train_1d_1, y_train)
params_1d = round(lr_1d.get_params(), 3)
y_pred_1d_1 = lr_1d.predict(X_test_1d_1)
for i in range(0, 6):
    print(X_train_1d_1.columns[i], ":", params_1d.values[i])

Quant : 133.45
10percentage : 899.534
12percentage : 1146.371
Logical : 149.242
collegeGPA : 1307.67
English : 136.434


In [1412]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình my_best_model
mae_1d_1 = mae(y_test, y_pred_1d_1)
print("Độ lỗi MAE trung bình", round(mae_1d_1, 3))

Độ lỗi MAE trung bình 104621.272


Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 
133.45 \times \text{Quant}
+ 899.534 \times \text{10percentage}
+ 1146.371 \times \text{12percentage}
+ 149.242 \times \text{Logical}
+ 1307.67 \times \text{collegeGPA}
+ 136.434 \times \text{English}
- 89598.216 \times \text{CollegeTier}
+ 16709.546 \times \text{Degree}
- 23151.196 \times \text{Gender}
+ 85.302 \times \text{ComputerProgramming}$$
$$\text{MAE} = 104621.272$$